In [1]:
import pandas as pd
import requests as req
import json
import time,os
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()
header = {"X-API-Key" : os.getenv('API_key')}

In [2]:
df = pd.read_csv('D:\\Air_Quality_health_Project\\data\\loaction_sensors.csv')
df.head()

,Unnamed: 0,Location_id,Sensor_id,Area,City,Sensor_name,Latitude,Longitude
0,0,12,23,SPARTAN,Kanpur,PM2.5,26.519000,80.233000
1,1,13,13866,Delhi Technological University,Delhi,NO₂ mass,28.744000,77.120000
2,2,13,13864,Delhi Technological University,Delhi,PM2.5,28.744000,77.120000
3,3,17,392,R K Puram,Delhi,CO mass,28.563262,77.186937
4,4,17,36,R K Puram,Delhi,NO₂ mass,28.563262,77.186937


In [ ]:
sensors_in_range = []
index = 0

In [ ]:
sensors_ids = df['Sensor_id'].sort_values()
sensors_ids

530    12236029
533    12236030
691    12236360
694    12236361
710    12236375
         ...   
730    12442196
851    12708245
852    12708246
885    12815121
886    12815122
Name: Sensor_id, Length: 99, dtype: int64

In [ ]:
for sensor in sensors_ids:
    try:
        print(f'{index} ', end = '')
        r = req.get(url= f'https://api.openaq.org/v3/sensors/{sensor}?limit=100&page=1',headers=header)
        if r.status_code == 200:
            request = r.json()['results']
            try:
                year_start = int(request[0]['coverage']['datetimeFrom']['local'].split('-')[0])
                year_end = int(request[0]['coverage']['datetimeTo']['local'].split('-')[0])
                # print(year_start, year_end)
                if year_start<=2020 and year_end>= 2021:
                    print(f'Sensor : {sensor} Starting year : {year_start} Ending Year : {year_end}')
                    sensors_in_range.append(sensor)
            except:
                print('Error on fetching data!!')
        else :
            print(f"Sensor : {sensor} invalid error code : {r.status_code}")
        time.sleep(1)
    except Exception as e:
        print("error" , e)        
    index+=1

800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866 867 868 869 870 871 872 873 874 875 876 877 878 879 880 Sensor : 12242120 invalid error code : 500
881 Sensor : 12242121 invalid error code : 500
882 Sensor : 12242129 invalid error code : 500
883 Sensor : 12242163 invalid error code : 500
884 Sensor : 12242164 invalid error code : 500
885 Sensor : 12243007 invalid error code : 500
886 Sensor : 12243008 invalid error code : 500
887 Sensor : 12243929 invalid error code : 500
888 Sensor : 12243930 invalid error code : 500
889 Sensor : 12243948 invalid error code : 500
890 Sensor : 12243949 invalid error code : 500
891 Sensor : 12252657 invalid error code : 500
892 Sensor : 12252658 invalid error code : 500
893 Sensor : 12442195 invalid error code : 500
894 Sensor : 12442196 

In [32]:
len(sensors_in_range)


435

In [6]:
sensors_in_range = pd.read_csv('seri.csv')
sensors_in_range = sensors_in_range['0']

In [7]:
df_inrange = (df[df['Sensor_id'].isin(sensors_in_range)]).reset_index(drop=True)
df_inrange.head()

,Unnamed: 0,Location_id,Sensor_id,Area,City,Sensor_name,Latitude,Longitude
0,33,407,710,Zoo Park,Hyderabad,CO mass,17.349694,78.451437
1,34,407,712,Zoo Park,Hyderabad,NO₂ mass,17.349694,78.451437
2,35,407,715,Zoo Park,Hyderabad,PM10,17.349694,78.451437
3,38,407,714,Zoo Park,Hyderabad,PM2.5,17.349694,78.451437
4,39,407,713,Zoo Park,Hyderabad,SO₂ mass,17.349694,78.451437


In [10]:

header = {"X-API-Key" : os.getenv("API_KEY")}
dataset = []
for sensor in sensors_in_range[2:5]:
    page = 1
    print('getting data for :', sensor)
    while True:
        try:
            request = req.get(f'https://api.openaq.org/v3/sensors/{sensor}/measurements/daily?limit=1000&page={page}&datetimeFrom=2024-01-01&datetimeTo=2025-01-01', headers =header)
            raw = request.json()
            data = raw['results']

            if type(raw['meta']['found']) == int :
                for param in data:
                    if '2020' in param['period']['datetimeFrom']['local']:
                        dataset.append([sensor, param['value'], param['period']['datetimeFrom']['local'].split("T")[0], param['period']['datetimeTo']['local'].split('T')[0]])
                    
                break
            else:
                for param in data:
                    if '2020' in param['period']['datetimeFrom']['local']:
                        dataset.append([sensor, param['value'], param['period']['datetimeFrom']['local'].split("T")[0], param['period']['datetimeTo']['local'].split('T')[0]])
                page+=1
                    
        except Exception as e:
            print('error in fetching from sensor ', sensor, 'Error : ', e)
        
        
        
dataset_df = pd.DataFrame(dataset, columns=['Sensor_id', 'Value', 'DateTimeFrom', 'DateTimeTo'])

getting data for : 713
getting data for : 714
getting data for : 715


In [ ]:
dataset_df.to_csv((os.path.join"test", 'measure.csv'), mode="a",index=False)

In [ ]:
(df_inrange.head(2)).merge(dataset_df, on='Sensor_id')

Location_id     0
Sensor_id       0
Area            0
City            0
Sensor_name     0
Latitude        0
Longitude       0
Value           0
DateTimeFrom    0
DateTimeTo      0
dtype: int64

In [7]:
df = pd.read_csv('D:\\Air_Quality_health_Project\\data\\measurments.csv')

In [18]:
df = df[~(df['Sensor_id'] == 'Sensor_id')].reset_index(drop=True)

In [24]:
df.to_csv('D:\\Air_Quality_health_Project\\data\\measurements.csv',index=False)